In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [40]:
import sys
import os
import json
import torch
from torch import tensor
import yaml
import numpy as np
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
from copy import deepcopy 
from PIL import Image
import torchvision.transforms as transforms
import torch.nn.functional as F
from scipy.spatial.transform import Rotation 
from nerfstudio.utils.eval_utils import eval_setup
from inerf.inerf_trainer import INerfTrainer
from inerf.inerf_utils import get_corrected_pose, load_eval_image_into_pipeline, get_relative_pose, get_absolute_diff_for_pose

In [3]:
os.chdir('/workspace')
MODEL_PATH = "/workspace/outputs/jackal_training_data_1/plane-nerf/2024-01-14_115715"
EVAL_PATH = "/stored_data/jackal_evaluation_data"
TRANSFORM_PATH = os.path.join(EVAL_PATH, "transforms.json")
with open(TRANSFORM_PATH) as f:
    TRANSFORM_JSON = json.load(f)

In [4]:
config_path = os.path.join(MODEL_PATH, "config.yml")
config, pipeline, checkpoint_path, _ = eval_setup(
                        Path(config_path),
                        test_mode="inference",
                    )

WARNING: Using slower TCNN CutlassMLP instead of TCNN FullyFusedMLP

Use layer width of 16, 32, 64, or 128 to use the faster TCNN FullyFusedMLP.

WARNING: Using slower TCNN CutlassMLP instead of TCNN FullyFusedMLP

Use layer width of 16, 32, 64, or 128 to use the faster TCNN FullyFusedMLP.

Loading latest checkpoint from load_dir

✅ Done loading checkpoint from 
outputs/jackal_training_data_1/plane-nerf/2024-01-14_115715/nerfstudio_models/step-000009999.ckpt

In [5]:
z = 2.5
pitch = 0.785
r = -2.5
init_tf = np.eye(4)
init_tf[:3, :3] = Rotation.from_rotvec(np.array([pitch, 0, 0])).as_matrix()
init_tf[0, 3] = -r
init_tf[2, 3] = z

In [6]:
pipeline = load_eval_image_into_pipeline(pipeline,EVAL_PATH,starting_pose=init_tf)
custom_camera_optimizer = deepcopy(pipeline.model.camera_optimizer)
custom_camera_optimizer.num_cameras = len(pipeline.datamanager.train_dataset.cameras)
trainer = INerfTrainer(config)
trainer.pipeline = pipeline
trainer.pipeline.model.camera_optimizer = custom_camera_optimizer
trainer.setup("inference")

/workspace/inerf/inerf/inerf_utils.py:100: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  camera_to_worlds = torch.cat([camera_to_worlds, tensor([tf]).float()], 0)


Setting up training dataset...

Caching all 300 images.

Output()

Loading latest Nerfstudio checkpoint from load_dir...


Done loading Nerfstudio checkpoint from 
outputs/jackal_training_data_1/plane-nerf/2024-01-14_115715/nerfstudio_models/step-000009999.ckpt

In [7]:
ground_truth_poses = []
for _ , batch in pipeline.datamanager.fixed_indices_train_dataloader:
    tf = TRANSFORM_JSON["frames"][int(batch['image_idx'])]["transform_matrix"]
    tf = np.asarray(tf)
    tf = tf[:3, :4 ]
    ground_truth_poses.append(tf)
ground_truth_poses = torch.tensor(ground_truth_poses)

In [8]:
n = 10
for i in range(10):
    for j in range(n):
        trainer.train_iteration_inerf(i*n + j)
    corrected_pose = get_corrected_pose(trainer)
    R = corrected_pose.cpu().detach()[0,:3,:3]
    t = corrected_pose.cpu().detach()[0,:3,3]
    rpy = Rotation.from_matrix(R).as_euler('xyz', degrees=True)
    print("Translation: ", t)
    print("Rotation: ", rpy)

Translation:  tensor([ 2.5005, -0.0086,  2.4977])
Rotation:  [ 4.52073562e+01  2.35932408e-02 -1.64198591e-01]
Translation:  tensor([ 2.4997, -0.0085,  2.4957])
Rotation:  [45.19307456  0.06547239 -0.13200478]
Translation:  tensor([ 2.5030, -0.0128,  2.5000])
Rotation:  [ 4.51844090e+01 -1.50405631e-02 -2.09000180e-01]
Translation:  tensor([ 2.5039, -0.0166,  2.5020])
Rotation:  [ 4.51641413e+01 -3.22469756e-02 -2.28376283e-01]
Translation:  tensor([ 2.5058, -0.0198,  2.5042])
Rotation:  [45.14832546 -0.06738836 -0.26416467]
Translation:  tensor([ 2.5074, -0.0229,  2.5065])
Rotation:  [45.12327166 -0.10039196 -0.29803314]
Translation:  tensor([ 2.5099, -0.0251,  2.5095])
Rotation:  [45.10634841 -0.16214491 -0.35728928]
Translation:  tensor([ 2.5129, -0.0285,  2.5123])
Rotation:  [45.08660404 -0.23007174 -0.42422377]
Translation:  tensor([ 2.5136, -0.0295,  2.5111])
Rotation:  [45.0609586  -0.22959402 -0.43073676]
Translation:  tensor([ 2.5148, -0.0309,  2.5107])
Rotation:  [45.0245333 

In [62]:
relative_pose = get_relative_pose(ground_truth_poses, corrected_pose)
t_diff, r_diff = get_absolute_diff_for_pose(relative_pose)

#Get averrage absolute translation and rotation error
print("Average translation error: ", torch.mean(t_diff))
print("Average rotation error: ", torch.mean(r_diff))

Average translation error:  tensor(3.5597, dtype=torch.float64)
Average rotation error:  tensor(1.5957, dtype=torch.float64)
